

# 1. Introduction

The objective of the assignemnt is to gather the relevant data from a patent website into a csv and json file. To achieve this, the following needs to be extracted from the document:

1) Grant ID 
<br>2) Patent Title
<br>3) Kind
<br>4) Number of Claims
<br>5) Inventors
<br>6) Citations Application Count
<br>7) Citations Examiners Count
<br>8) Claims Text
<br>9) Abstract

Utilizing regular expressions the data is retrieved from the document and placed into a Data Frame which will be covered more in details below.


 

 Libraries Used

For the purpose of the assigment only Pandas and Re were used. 

In [1]:
import pandas as pd
import re

#  2. Loading Data

To start, the file is read into the Jupiter Notebook and each line is copied into a data frame which we define as "df". Each line is written in a different row which we define the column as "raw," it's from where all of the relevant data will be taken into other columns.
 
Basic concept behind out code:
Our concept is to use regular expressions to extract the data from the "raw" column into the following columns:

raw ---> Grant ID
raw ---> Property (patent tittle, patent kind, claims, author, etc)
raw ---> Value (Relevant information that describes the property, eg: patent tittle: "Fungicidal Substituted")

After all of the relevant data is gathered, the table will be pivoted making the "Property" the tittle for all of the columns and the "raw" column will be deleted.


In [2]:
# The file is read into the working directory with sep as '`' since this character forces everything into a single column
df = pd.read_csv('data.txt', names=['raw'], encoding="cp1252", sep="`")

# Gets the grant id for all of the patents
df['grant_id'] = df['raw'].str.extract("file=\"(.*?)-")

# will drop all empty lines and pads the grant ids to fill the column
df = df.dropna(how='all')
df = df.fillna(method='pad')

# Uneeded data is droped 
df = df[~df['raw'].str.contains("<?xml")]

# 2 new columns are created, they will contain all of the extracted data
df['property'] = None
df['value'] = None
df.head()

,raw,grant_id,property,value
1,"<us-patent-grant lang=""EN"" dtd-version=""v4.5 2...",US10360552,None,None
2,<us-bibliographic-data-grant>,US10360552,None,None
3,<publication-reference>,US10360552,None,None
4,<document-id>,US10360552,None,None
5,<country>US</country>,US10360552,None,None


# 3. Functions

GetProperty and getValue are probably the most important functions in the code. When the pattern established in the getProperty is met, in the property row it will assume the value of the pattern. For example: invention-title in raw column this will mean that in that row there's an invention title and therefor, a patent tittle.

For GetValue we used the regular expression to match certain text pattern which are:

1) Invention tittle: All of the text after ">" until the first "<" if the line contains the words "invention title"
<br>2) Number of Claims: Number after ">" if the line coints "number-of-claims"
<br>3) First Name: All letters after ">" until the first "<" if the line contains "first-name"
<br>4) Last Name: All letters after ">" until the first "<" if the line contains "last-name"
<br>5) Cited By Applicant: It will return a 1 as later on all the 1s will be added by a groupby function
<br>6) Cited by Examiner:It will return a 1 as later on all the 1s will be added by a groupby function
<br>7) Kind: All letters after ">" until the first "<" if the line contains "<kind"
<br>8) Claim Text: All letters after ">" until the first "<" "<claim text"

In [3]:
#function to get column names
def getProperty(rawStr):
    if 'invention-title' in rawStr:
        return 'patent_title'
    elif 'number-of-claims' in rawStr:
        return 'number_of_claims'
    elif 'first-name' in rawStr:
        return 'first_name'
    elif 'last-name' in rawStr:
        return 'last_name'
    elif 'cited by applicant' in rawStr:
        return 'citations_applicant_count'
    elif 'cited by examiner' in rawStr:
        return 'citations_examiner_count'
    elif '<kind>' in rawStr:
        return 'kind'
    elif '<claim-text>' in rawStr:
        return 'claims_text'
    elif '<claim id=' in rawStr:
        return 'claim_stub'
    elif '<inventor ' in rawStr:
        return 'inventor'
    elif '<primary-examiner' in rawStr:
        return 'primary-examiner'
    elif '<abstract id' in rawStr:
        return 'abstract'
    elif '</abstract>' in rawStr:
        return '</abstract>'
    
    return None
#Function to get the values for the columns
def getValue(rawStr):
    if 'invention-title' in rawStr:
        value = re.search(">(.*)<", rawStr)
        return value.group(1)
    elif 'number-of-claims' in rawStr:
        value = re.search(">(\d+)<", rawStr)
        return value.group(1)
    elif 'first-name' in rawStr:
        value = re.search(">(.*)<", rawStr)
        return value.group(1)
    elif 'last-name' in rawStr:
        value = re.search(">(.*)<", rawStr)
        return value.group(1)
    elif 'cited by applicant' in rawStr:
        return 1
    elif 'cited by examiner' in rawStr:
        return 1
    elif '<kind>' in rawStr:
        value = re.search(">(.*)<", rawStr)
        return value.group(1)
    elif '<claim-text' in rawStr:
        value = re.search('>(.*)<?', rawStr)
        value = value.group(1)
        htmlTagMatcher = re.compile('<.*?>')
        value = re.sub(htmlTagMatcher, '', value)
        return value
    return None

# 4. Extracting Data

First and most important, we call the functions in our data frame to apply all of the regular expressions defined above and fill out the information and get all of the relevant info.


In [4]:
#Define the values of the property and value columns for the rows that we need
df['property'] = df.apply(lambda row: getProperty(row['raw']), axis=1)
df['value'] = df.apply(lambda row: getValue(row['raw']), axis=1)
df.head()

,raw,grant_id,property,value
1,"<us-patent-grant lang=""EN"" dtd-version=""v4.5 2...",US10360552,None,None
2,<us-bibliographic-data-grant>,US10360552,None,None
3,<publication-reference>,US10360552,None,None
4,<document-id>,US10360552,None,None
5,<country>US</country>,US10360552,None,None


## Abstract

The first extraction is with the abstract, the abstract is done different than the other methods as the abstract can be in multiple lines. 

1) First the "property" values from the dataframe are set to "abstract" on the lines that contain </abstract> and using the pad filling method the abstract word is draged to fill Nulls.

2) tempDF is a new data frame that is created with only the rows on which the property column contains the word "abstract".

3) The text is grouped by the grant ID and property, this will make the text from the value column to append into the first row.

4) As it appends everything in multiple rows, there are many tags that need to be cleaned and we use a regular expression to substitute everything in between "<>"
    tempDf['value'].replace(regex=True,inplace=True,to_replace=r'<.*?>',value=r'')





In [5]:

df['temp'] = None
#creates a temporary column containing all of the abstract information
df.loc[((df['property']=='abstract') | (df['property']=='</abstract>')), 'temp'] = df['property']
df['temp'].fillna(method="pad", inplace=True)
df = df[~((df['property']=='abstract') | (df['property']=='</abstract>'))]
df.loc[(df['temp']=='abstract'), 'property'] = 'abstract'
df.loc[(df['temp']=='abstract'), 'value'] = df['raw']

#a new Data Frame is created to hold all of the abstract data in it
tempDf = df.copy()
tempDf = tempDf.drop(['raw','temp'], 1)
tempDf = tempDf[(tempDf['property']=='abstract')]
#The abstract information is grouped by appending all of the abstract into 1 row
tempDf['temp'] = tempDf.groupby(['grant_id', 'property'])['value'].transform(lambda x: ''.join(str(y) for y in x))
tempDf['value'] = tempDf['temp']
tempDf = tempDf.drop('temp', 1)
tempDf = tempDf.drop_duplicates(keep='first')
tempDf['value'] = tempDf['value'].astype(str)
#tags are removed from the appended text
tempDf['value'].replace(regex=True,inplace=True,to_replace=r'<.*?>',value=r'')


df = df[~(df['property']=='abstract')]

## Citations Count

As it was previously established, Citations applicant count and examiners count where given a 1. Utilizing the grouby function all of the citations are summed together and grouped by grant id and property, this will get us the amount of citations per gant.

In [6]:
df['temp'] = None
#Groupby function allow us to group all of the claim text and add them up
df['temp'] = df.groupby(['grant_id', 'property'])['value'].transform('sum')
df.loc[(df['property']=='citations_applicant_count') | (df['property']=='citations_examiner_count'), 'value'] = df['temp']
#TEmporary column is droped since it's not needed anymore
df = df.drop('temp', 1)
df.head()

,raw,grant_id,property,value
1,"<us-patent-grant lang=""EN"" dtd-version=""v4.5 2...",US10360552,None,None
2,<us-bibliographic-data-grant>,US10360552,None,None
3,<publication-reference>,US10360552,None,None
4,<document-id>,US10360552,None,None
5,<country>US</country>,US10360552,None,None


## Get Patent Kind

Patents can have different kinds for example: A1, A2, B1, etc. Therefore, it is needed to search for that specific value in the text. 
A new temporary column is created containing "<publication-reference" this will help us to select the only kinds text we need as there are different kinds making reference to other documents so this will make unique the kind in reference the patent.

A dictionary named patent_dict was created containing all of the patent kinds as a key and the description as a value. After finding the key in the dictionary the value will be obtained and this will be our output for the dataframe.



After having the correct kind of the patent, the function getkind_text is applied to obtain the text description and the temp column is droped.




In [7]:
df['temp'] = None
# A new column is created which contains the kind abreviation
df.loc[(df['raw'].str.contains("<publication-reference>")), 'temp'] = '<publication-reference>'
df.loc[(df['raw'].str.contains("</publication-reference>")), 'temp'] = '</publication-reference>'

#In order to get the correct kind, 2 criteria needs to be met. Have property "kind" and ""</publication-ref" in the temp column
df['temp'].fillna(method="pad", inplace=True)
df = df[~((df['property']=='kind') & (df['temp']=='</publication-reference>'))]

#Function containing the kind description in a dictionary
def getkind_text (value_str):
        patent_dict = {"X0" : "The original filing application number of a pending application that has been subsequently published or granted/issued.", "A1" : "Utility Patent Grant issued prior to January 2, 2001.",  "A2" : "Second or subsequent publication of a Utility Patent Application.", "B1": "Utility Patent Grant (no published application) issued on or after January 2, 2001.", "B2" : "Utility Patent Grant (with a published application) issued on or after January 2, 2001.", "P1" : "Plant Patent Grant issued prior to January 2, 2001.","P2" : "Plant Patent Grant (no published application) issued on or after January 2, 2001.", "P3" : "Plant Patent Grant (with a published application) issued on or after January 2, 2001.", "P4" : "Second or subsequent publication of a Plant Patent Application.", "S1": "Design Patent" , "E1" : "Reissue Patent"}
        if value_str in patent_dict:
            return patent_dict[value_str]
        return value_str

# The function is applied get the kind description
df['value'] = df.apply(lambda row: getkind_text(row['value']), axis=1)
df = df.drop('temp', 1)

## Inventor's Name

We noticed that there were names of two types: inventors and primary-examiners. In order to get all the first and last names for inventors, we first fetched all the first and last names using the getValue function above. Next, we identified all first and last names based on whether they are for inventors or primary-examiners using the raw column and padded it to fill all null values and correspond with first and last names. Then, we dropped all names that were not for inventors and cleaned up the unnecessary columns.



In [8]:
df['temp'] = None


df = df[~((df['property'].isna()))]

df.loc[(df['property']=='inventor'), 'temp'] = 'inventor'
df.loc[(df['property']=='primary-examiner'), 'temp'] = 'primary-examiner'

df['temp'].fillna(method="pad", inplace=True)

df = df[~((df['property']=='inventor') | (df['property']=='primary-examiner'))]

df = df[~(((df['property']=='first_name') | ((df['property']=='last_name'))) & (df['temp']!='inventor'))]

df = df.drop('temp', 1)
df = df.dropna(how='all')

df['temp'] = None
df['temp'] = df.groupby(['grant_id', 'property'])['value'].transform(lambda x: '?'.join(str(y) for y in x))

df.loc[(df['property']=='last_name') | (df['property']=='first_name'), 'value'] = df['temp']
df = df.drop('temp', 1)

## Getting Claims Text

We noticed that a patent can have multiple claims. Claims, within themselves, were appended as is while two individual claims were appended with a space between them.

A function is created named "getClaimIds" this will append into a new temporary column all of the lines containing "<claim id" up to "</claims>". This will separate all of the claims in which we will later group.

Using this claim-id, we identify what rows are to be appended as is and which rows are to be appended with a space. All these claims are then aggregated into a single row containing all the claim-text values.

Here, we only join individual claims for a particular patent. We aggregate all the claims for a particular patent in the clean-up step.

In [9]:
df['temp'] = None

def getClaimIds(rawStr):
    if '<claim id=' in rawStr:
        claim_id = re.search(r'<claim id=\"(.*?)\"', rawStr)
        if claim_id:
            return claim_id.group(1)
    elif '</claims>' in rawStr:
        return 'claims-end'
    return None

df['temp'] = df.apply(lambda row: getClaimIds(row['raw']), axis=1)

df['temp'].fillna(method="pad", inplace=True)

df = df[~(df['property'] == 'claim_stub')]
df.loc[(df['temp'] == 'claims-end'), 'temp'] = None
df = df[~((df['property'].isna()))]

df['temp'].fillna(0, inplace=True)
df['temp2'] = df.groupby(['grant_id', 'temp', 'property'])['value'].transform(lambda x: ''.join(str(y) for y in x))

df.loc[(df['property']=='claims_text'), 'value'] = df['temp2']

df = df.drop(['temp2', 'temp'], 1)

# Final clean-up

Once all the aggregated properties are completed, we then begin the clean up tasks before the table is pivoted.

We drop the 'raw' column first and then drop all duplicate rows.

Then, all the claims for a particular patent are aggregated into a single row and duplicates are dropped.

In [10]:
df = df.drop('raw', 1)

df['temp'] = None

df = df.drop_duplicates(keep='first')

df['temp'] = df.groupby(['grant_id', 'property'])['value'].transform(lambda x: '[' + ','.join(str(y) for y in x) + ']')
df.loc[(df['property']=='claims_text'), 'value'] = df['temp']

df = df.drop_duplicates(keep='first')

df = df.drop('temp', 1)

# 5. Pivoting Data Frame

Here is where we join the temporarily created dataframe to our intended output dataframe and then pivot the table. Finaly the table is pivoted to have distict attributes in different columns

In [11]:
#main df and tempdf are concatenated to get 1 data frame
df = pd.concat([df, tempDf])
df = df.drop_duplicates(keep='first')
df = df.pivot('grant_id', 'property', 'value')

In [12]:
df['citations_applicant_count'].fillna(0, inplace=True)
df['citations_examiner_count'].fillna(0, inplace=True)
df['abstract'].fillna('NA', inplace=True)

In [13]:
def get_full_name(first_name_list_str, last_name_list_str):
    try:
        first_name_list = first_name_list_str.split('?')
        last_name_list = last_name_list_str.split('?')

        full_name_list = list()

        for i in list(range(0, len(first_name_list))):
            try:
                full_name_list.append(first_name_list[i] + ' ' + last_name_list[i])
            except:
                print(first_name_list_str)
                print(last_name_list_str)
                print(len(first_name_list), ' ', len(last_name_list))

        return '[' + ','.join(full_name_list) + ']'
    except:
        print(first_name_list_str)

In [14]:
df['inventors'] = None

df['inventors'] = df.apply(lambda row: get_full_name(row['first_name'], row['last_name']), axis=1)
df = df.drop(['first_name', 'last_name'], 1)

The data frame titles are adjusted to make sure they match the sample ouput.

In [15]:
df = df[['patent_title', 'kind', 'number_of_claims', 'inventors', 'citations_applicant_count', 'citations_examiner_count', 'claims_text', 'abstract']]

# 6. CSV

A CSV file can be easily created using 1 line of code that is the following

In [16]:
df.to_csv('csv_file.csv', sep=',', encoding='utf-8')

# 7. JSON

Json file has a distinct pattern in the structure. Therefore, we used a for loop to iterate each row of our dataframe adding the corresponding pattern. Each time the program iterates through 1 column it will append the corresponding tags into it

In [17]:
json_output = ''
#iteration to have the correct JSON format
for index, row in df.iterrows():
    json_row = ''
    json_row += '"' + index + '":{'
    json_row += '"patent_title":"' + row['patent_title'] + '",'
    json_row += '"kind":"' + row['kind'] + '",'
    json_row += '"number_of_claims":' + str(row['number_of_claims']) + ','
    json_row += '"inventors":"' + row['inventors'] + '",'
    json_row += '"citations_applicant_count":' + str(row['citations_applicant_count']) + ','
    json_row += '"citations_examiner_count":' + str(row['citations_examiner_count']) + ','
    json_row += '"claims_text":"' + row['claims_text'] + '",'
    json_row += '"abstract":"' + row['abstract'] + '"'
    json_row += '},'
    
    json_output += json_row

json_output = json_output[:-1]
json_output = '{' + json_output + '}'

In [18]:
json_output

'{"US10356978":{"patent_title":"Suspension system for lawnmower","kind":"Utility Patent Grant (no published application) issued on or after January 2, 2001.","number_of_claims":27,"inventors":"[Charles Bradley Covington,Warren Russell Long,Christopher L. Drymalla]","citations_applicant_count":50,"citations_examiner_count":2,"claims_text":"[1. A riding lawnmower system comprising:a frame;a right rear drive system comprising:a right integrated hydraulic transaxle comprising a right hydraulic pump, a right hydraulic motor and a right drive axle, the right drive axle configured to rotate about a right axle axis, and the right integrated hydraulic transaxle pivotally coupled to the frame such that the right integrated hydraulic transaxle is configured to pivot about a right pivot axis oriented transverse to the right axle axis;a right rear drive wheel coupled to the right drive axle of the right integrated hydraulic transaxle, the right rear drive wheel configured to rotate about the right 

In [19]:
outputJSONPipe = open('json_file.json', "w", encoding="utf-8")

In [20]:
outputJSONPipe.write(str(json_output))

1058301

# 8. Summary

1. We used Data Frame manipulation to extract the relevant data into different columns
2. With the aid of regular expressions we were able to extract and replace important information
3. Creating temporary columns and "workspaces" were the key for it to be succesfull